# 重新定义大模型和智能体

## 多轮对话

使用`messages`管理多轮对话的记忆。<br>
你也可以在`chat`函数中通过参数`k`限制希望保留的对话轮次，但这不影响`messages`记录完整的对话过程。

In [3]:
from textlong.llm import fake_llm, zhipu, qwen
from textlong.desk import Desk

d = Desk(zhipu)

### 默认记住10轮

In [4]:
d.chat("你能帮我写一首关于兔子做梦的四句儿歌?")

当然可以！以下是一首关于兔子做梦的儿歌：

小兔子，梦儿香，
跳跃在花丛中，
遇见彩虹桥，
快乐歌声绕耳旁。



[{'role': 'user', 'content': '你能帮我写一首关于兔子做梦的四句儿歌?'},
 {'role': 'assistant',
  'content': '当然可以！以下是一首关于兔子做梦的儿歌：\n\n小兔子，梦儿香，\n跳跃在花丛中，\n遇见彩虹桥，\n快乐歌声绕耳旁。'}]

In [3]:
d.chat("换成小羊吧")

好的，这是一首关于小羊做梦的四句儿歌：

梦里青山小羊唱，
碧草如茵梦中行。
梦中小溪潺潺流，
醒来草地上欢跳。



[{'role': 'user', 'content': '你能帮我写一首关于兔子做梦的四句儿歌?'},
 {'role': 'assistant',
  'content': '当然可以！以下是一首关于兔子做梦的四句儿歌：\n\n梦里花间跳兔子，\n草丛深处藏秘密。\n月亮做伴夜悠长，\n醒来阳光洒窗棂。'},
 {'role': 'user', 'content': '换成小羊吧'},
 {'role': 'assistant',
  'content': '好的，这是一首关于小羊做梦的四句儿歌：\n\n梦里青山小羊唱，\n碧草如茵梦中行。\n梦中小溪潺潺流，\n醒来草地上欢跳。'}]

### 限制记忆中的对话轮数

In [4]:
d.chat("改为在蓝天上")

蓝天白云小羊梦，
梦中翅膀轻轻扬。
云朵做床梦飞扬，
醒来蓝天笑洋洋。



[{'role': 'user', 'content': '你能帮我写一首关于兔子做梦的四句儿歌?'},
 {'role': 'assistant',
  'content': '当然可以！以下是一首关于兔子做梦的四句儿歌：\n\n梦里花间跳兔子，\n草丛深处藏秘密。\n月亮做伴夜悠长，\n醒来阳光洒窗棂。'},
 {'role': 'user', 'content': '换成小羊吧'},
 {'role': 'assistant',
  'content': '好的，这是一首关于小羊做梦的四句儿歌：\n\n梦里青山小羊唱，\n碧草如茵梦中行。\n梦中小溪潺潺流，\n醒来草地上欢跳。'},
 {'role': 'user', 'content': '改为在蓝天上'},
 {'role': 'assistant', 'content': '蓝天白云小羊梦，\n梦中翅膀轻轻扬。\n云朵做床梦飞扬，\n醒来蓝天笑洋洋。'}]

调用时可以切换模型，例如用来模拟和查看提示语的 `fake_llm`：

In [5]:
d.state.messages

[{'role': 'user', 'content': '你能帮我写一首关于兔子做梦的四句儿歌?'},
 {'role': 'assistant',
  'content': '当然可以！以下是一首关于兔子做梦的四句儿歌：\n\n梦里花间跳兔子，\n草丛深处藏秘密。\n月亮做伴夜悠长，\n醒来阳光洒窗棂。'},
 {'role': 'user', 'content': '换成小羊吧'},
 {'role': 'assistant',
  'content': '好的，这是一首关于小羊做梦的四句儿歌：\n\n梦里青山小羊唱，\n碧草如茵梦中行。\n梦中小溪潺潺流，\n醒来草地上欢跳。'},
 {'role': 'user', 'content': '改为在蓝天上'},
 {'role': 'assistant', 'content': '蓝天白云小羊梦，\n梦中翅膀轻轻扬。\n云朵做床梦飞扬，\n醒来蓝天笑洋洋。'}]

In [6]:
d.chat("改为在水塘", llm=zhipu, k=1)

水塘碧波荡漾间，
蓝天倒影如镜悬。
荷叶轻摇鱼儿跃，
蓝天白云共悠然。



[{'role': 'user', 'content': '改为在蓝天上'},
 {'role': 'assistant', 'content': '蓝天白云小羊梦，\n梦中翅膀轻轻扬。\n云朵做床梦飞扬，\n醒来蓝天笑洋洋。'},
 {'role': 'user', 'content': '改为在水塘'},
 {'role': 'assistant', 'content': '水塘碧波荡漾间，\n蓝天倒影如镜悬。\n荷叶轻摇鱼儿跃，\n蓝天白云共悠然。'}]

### 多轮对话 + 回调工具

In [1]:
from textlong.llm.tools import tool, convert_to_openai_tool
from textlong.llm import qwen
from textlong.desk import Desk
import json

@tool
def get_current_weather(location: str):
    """获取城市的天气情况"""
    return f"{location}今天是晴天。 "

messages = []
d = Desk(qwen, toolkits=[get_current_weather], tools=[convert_to_openai_tool(get_current_weather)])

In [2]:
print(d.toolkits)

[StructuredTool(name='get_current_weather', description='获取城市的天气情况', args_schema=<class 'pydantic.main.get_current_weatherSchema'>, func=<function get_current_weather at 0x10c24bbe0>), StructuredTool(name='python_code', description='回答关于的数据查询和分析的问题。\n具体包括：', args_schema=<class 'textlong.llm.tools.python_code.create_python_code_tool.<locals>.PythonCodeInput'>, func=<function create_python_code_tool.<locals>.python_code at 0x10c24ac20>)]


In [3]:
d.chat("今天广州天气如何啊")

>-[TOOLS_CALL]>> {"index": 0, "id": "call_e85d1cf3a6b44c449f0d2f", "type": "function", "function": {"name": "get_current_weather", "arguments": ""}}
>-[TOOLS_CALL]>> {"index": 0, "id": "", "type": "function", "function": {"arguments": "{\"location\": \""}}
>-[TOOLS_CALL]>> {"index": 0, "id": "", "type": "function", "function": {"arguments": "广州\"}"}}
>-[TOOLS_CALL]>> {"index": 0, "id": "", "type": "function", "function": {}}
{"0": {"index": 0, "id": "call_e85d1cf3a6b44c449f0d2f", "type": "function", "function": {"name": "get_current_weather", "arguments": "{\"location\": \"广州\"}"}}}
>-[TOOL_RESP]>> 广州今天是晴天。 
广州今天的天气是晴朗的。



[{'role': 'user', 'content': '今天广州天气如何啊'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'index': 0,
    'id': 'call_e85d1cf3a6b44c449f0d2f',
    'type': 'function',
    'function': {'name': 'get_current_weather',
     'arguments': '{"location": "广州"}'}}]},
 {'role': 'tool', 'name': 'get_current_weather', 'content': '广州今天是晴天。 '},
 {'role': 'assistant', 'content': '广州今天的天气是晴朗的。'}]

In [16]:
d.chat("这是怎么得出来的？")

我调用了get_current_weather这个工具来查询广州的天气情况，它返回的结果表明广州今天是晴天。请注意，实际天气可能会有所变化，具体以当地气象部门发布的预报为准。



[{'role': 'user', 'content': '今天广州天气如何啊'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'index': 0,
    'id': 'call_31de2f22fb504d3f8ac574',
    'type': 'function',
    'function': {'name': 'get_current_weather',
     'arguments': '{"location": "广州"}'}}]},
 {'role': 'tool', 'name': 'get_current_weather', 'content': '广州今天是晴天。 '},
 {'role': 'assistant', 'content': '广州今天的天气是晴朗的。'},
 {'role': 'user', 'content': '这是怎么得出来的？'},
 {'role': 'assistant',
  'content': '我调用了get_current_weather这个工具来查询广州的天气情况，它返回的结果表明广州今天是晴天。请注意，实际天气可能会有所变化，具体以当地气象部门发布的预报为准。'}]

In [5]:
d.chat("再看看上海")

>-[TOOLS_CALL]>> {"index": 0, "id": "call_ae91422dcf734f7f8c5fbd", "type": "function", "function": {"name": "get_current_weather", "arguments": ""}}
>-[TOOLS_CALL]>> {"index": 0, "id": "", "type": "function", "function": {"arguments": "{\"location\": \""}}
>-[TOOLS_CALL]>> {"index": 0, "id": "", "type": "function", "function": {"arguments": "上海\"}"}}
>-[TOOLS_CALL]>> {"index": 0, "id": "", "type": "function", "function": {}}
{"0": {"index": 0, "id": "call_ae91422dcf734f7f8c5fbd", "type": "function", "function": {"name": "get_current_weather", "arguments": "{\"location\": \"上海\"}"}}}
>-[TOOL_RESP]>> 上海今天是晴天。 
查询了上海的天气，结果显示今天上海也是晴天。



[{'role': 'user', 'content': '今天广州天气如何啊'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'index': 0,
    'id': 'call_48fef997b25447699c9f66',
    'type': 'function',
    'function': {'name': 'get_current_weather',
     'arguments': '{"location": "广州"}'}}]},
 {'role': 'tool', 'name': 'get_current_weather', 'content': '广州今天是晴天。 '},
 {'role': 'assistant', 'content': '广州今天的天气是晴朗的。'},
 {'role': 'user', 'content': '这是怎么得出来的？'},
 {'role': 'assistant',
  'content': '我调用了 get_current_weather 这个工具函数来查询广州的天气，它返回的结果表明广州今天是晴天。请注意，实际使用中该函数应能提供更详尽的信息，例如温度、湿度等，但在这个模拟环境中，它仅返回了天气状况。'},
 {'role': 'user', 'content': '再看看上海'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'index': 0,
    'id': 'call_ae91422dcf734f7f8c5fbd',
    'type': 'function',
    'function': {'name': 'get_current_weather',
     'arguments': '{"location": "上海"}'}}]},
 {'role': 'tool', 'name': 'get_current_weather', 'content': '上海今天是晴天。 '},
 {'role': 'assistant', 'content': '查询了上海的天气，结果显示今天上海也是晴天。'}]

In [6]:
d.state.messages

[{'role': 'user', 'content': '今天广州天气如何啊'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'index': 0,
    'id': 'call_48fef997b25447699c9f66',
    'type': 'function',
    'function': {'name': 'get_current_weather',
     'arguments': '{"location": "广州"}'}}]},
 {'role': 'tool', 'name': 'get_current_weather', 'content': '广州今天是晴天。 '},
 {'role': 'assistant', 'content': '广州今天的天气是晴朗的。'},
 {'role': 'user', 'content': '这是怎么得出来的？'},
 {'role': 'assistant',
  'content': '我调用了 get_current_weather 这个工具函数来查询广州的天气，它返回的结果表明广州今天是晴天。请注意，实际使用中该函数应能提供更详尽的信息，例如温度、湿度等，但在这个模拟环境中，它仅返回了天气状况。'},
 {'role': 'user', 'content': '再看看上海'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'index': 0,
    'id': 'call_ae91422dcf734f7f8c5fbd',
    'type': 'function',
    'function': {'name': 'get_current_weather',
     'arguments': '{"location": "上海"}'}}]},
 {'role': 'tool', 'name': 'get_current_weather', 'content': '上海今天是晴天。 '},
 {'role': 'assistant', 'content': '查询了上海的天气，结果显示今天上海也是晴天。'}]

## 写作提示语模板

In [7]:
from textlong.hub import load_chat_template, load_resource_chat_template

### 加载对话模板的资源

指定资源库中的模板ID即可加载模板。

In [8]:
load_chat_template("IDEA")

PromptTemplate(input_variables=['task'], partial_variables={'knowledge__': ''}, template='你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n{{#knowledge__}}\n\n**你必须参考这些已有资料：**\n{{knowledge__}}\n\n{{/knowledge__}}\n**你的任务是:**\n{{task}}\n', template_format='mustache')

In [9]:
load_chat_template("OUTLINE")

PromptTemplate(input_variables=['task'], partial_variables={'knowledge__': ''}, template='你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n3. 请按照需要输出一份写作提纲\n4. 提纲的构成应当由多个阔系额要求构成\n5. 扩写要求必须包含清晰描述，如预估字数、创意要点、创作思路、创作中涉及到的实体名称等\n5. 每一个扩写要求由完整的 `<OUTLINE>`和`</OUTLINE>` 包裹\n6. 你只能输出提纲，不要输出具体的扩写内容\n\n**你的任务是:**\n{{task}}\n\n{{#knowledge__}}\n\n**你必须参考这些已有资料：**\n{{knowledge__}}\n{{/knowledge__}}\n\n\n**输出例子1**\n```\n# 第一章 标题1\n## 一、XXXX\n<OUTLINE>\n扩写摘要:\n对扩写内容做摘要总结\n\n扩写要求：\n- xxx\n- xxx\n</OUTLINE>\n\n## 二、XXXX\n<OUTLINE>\n</OUTLINE>\n\n## （更多扩写提纲）\n```\n\n', template_format='mustache')

### 查看构造出的提示语模板

In [10]:
print(load_chat_template("OUTLINE").format(task="请帮我写一首歌"))

你是强大的写作助手。

你必须遵循以下约束来完成任务:
1. 直接输出你的结果，不要评论，不要啰嗦
2. 使用markdown格式输出
3. 请按照需要输出一份写作提纲
4. 提纲的构成应当由多个阔系额要求构成
5. 扩写要求必须包含清晰描述，如预估字数、创意要点、创作思路、创作中涉及到的实体名称等
5. 每一个扩写要求由完整的 `<OUTLINE>`和`</OUTLINE>` 包裹
6. 你只能输出提纲，不要输出具体的扩写内容

**你的任务是:**
请帮我写一首歌



**输出例子1**
```
# 第一章 标题1
## 一、XXXX
<OUTLINE>
扩写摘要:
对扩写内容做摘要总结

扩写要求：
- xxx
- xxx
</OUTLINE>

## 二、XXXX
<OUTLINE>
</OUTLINE>

## （更多扩写提纲）
```




### 单轮写作

In [1]:
from textlong.llm import qwen, fake_llm
from textlong.desk import Desk

d = Desk(qwen)

In [2]:
d.write("帮我写一首关于兔子的四句儿歌", template="IDEA")
print(d.output)

小白兔白又白，  
耳朵长长跳得欢。  
爱吃萝卜和青菜，  
蹦蹦跳跳真可爱。

小白兔白又白，  
耳朵长长跳得欢。  
爱吃萝卜和青菜，  
蹦蹦跳跳真可爱。




In [9]:
d.chat("它喜欢吃什么？", k=1, llm=fake_llm)

>-[INFO]>> system: 你是强大的写作助手。

你必须遵循以下约束来完成任务:
1. 直接输出你的结果，不要评论，不要啰嗦
2. 使用markdown格式输出
**你的任务是:**
帮我写一首关于兔子的四句儿歌

>-[INFO]>> user: 请开始
>-[INFO]>> assistant: 小白兔白又白，  
耳朵长长跳得欢。  
爱吃萝卜和青菜，  
蹦蹦跳跳真可爱。
>-[INFO]>> user: 它喜欢吃什么？
>-[INFO]>> assistant: 这是一个模拟调用!
>-[INFO]>> user: 它喜欢吃什么？
>-[INFO]>> assistant: 这是一个模拟调用!
>-[INFO]>> user: 它喜欢吃什么？
这是一个模拟调用!



[{'role': 'system',
  'content': '你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n**你的任务是:**\n帮我写一首关于兔子的四句儿歌\n'},
 {'role': 'user', 'content': '请开始'},
 {'role': 'assistant',
  'content': '小白兔白又白，  \n耳朵长长跳得欢。  \n爱吃萝卜和青菜，  \n蹦蹦跳跳真可爱。'},
 {'role': 'user', 'content': '它喜欢吃什么？'},
 {'role': 'assistant', 'content': '这是一个模拟调用!'},
 {'role': 'user', 'content': '它喜欢吃什么？'},
 {'role': 'assistant', 'content': '这是一个模拟调用!'},
 {'role': 'user', 'content': '它喜欢吃什么？'},
 {'role': 'assistant', 'content': '这是一个模拟调用!'}]

In [10]:
d.state.messages

[{'role': 'system',
  'content': '你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n**你的任务是:**\n帮我写一首关于兔子的四句儿歌\n'},
 {'role': 'user', 'content': '请开始'},
 {'role': 'assistant',
  'content': '小白兔白又白，  \n耳朵长长跳得欢。  \n爱吃萝卜和青菜，  \n蹦蹦跳跳真可爱。'},
 {'role': 'user', 'content': '它喜欢吃什么？'},
 {'role': 'assistant', 'content': '这是一个模拟调用!'},
 {'role': 'user', 'content': '它喜欢吃什么？'},
 {'role': 'assistant', 'content': '这是一个模拟调用!'},
 {'role': 'user', 'content': '它喜欢吃什么？'},
 {'role': 'assistant', 'content': '这是一个模拟调用!'},
 {'role': 'user', 'content': '它喜欢吃什么？'},
 {'role': 'assistant', 'content': '这是一个模拟调用!'},
 {'role': 'user', 'content': '它喜欢吃什么？'},
 {'role': 'assistant', 'content': '这是一个模拟调用!'}]

In [4]:
d.from_outline()

>-[INFO]>> 没有提纲可供扩写


In [3]:
d.state.messages

[{'role': 'system',
  'content': '你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n**你的任务是:**\n帮我写一首关于兔子的四句儿歌\n'},
 {'role': 'user', 'content': '请开始'},
 {'role': 'assistant',
  'content': '小白兔白又白，  \n耳朵长长跳得欢。  \n爱吃萝卜和青菜，  \n蹦蹦跳跳真可爱。'}]

### 单轮写作 + 回调工具

In [11]:
from textlong.llm.tools import tool, convert_to_openai_tool
from textlong.llm import qwen
from textlong.desk import Desk
import json

@tool
def get_current_weather(location: str):
    """获取城市的天气情况"""
    return f"{location}今天下大雨。 "

d = Desk(qwen, tools=[convert_to_openai_tool(get_current_weather)], toolkits=[get_current_weather])
d.write({"task": "根据广州今天的天气写一首4句歌词"}, template="IDEA")

>-[TOOLS_CALL]>> {"index": 0, "id": "call_b9cfb902adf647a4b6f2e6", "type": "function", "function": {"name": "get_current_weather", "arguments": ""}}
>-[TOOLS_CALL]>> {"index": 0, "id": "", "type": "function", "function": {"arguments": "{\"location\": \""}}
>-[TOOLS_CALL]>> {"index": 0, "id": "", "type": "function", "function": {"arguments": "广州\"}"}}
>-[TOOLS_CALL]>> {"index": 0, "id": "", "type": "function", "function": {}}
{"0": {"index": 0, "id": "call_b9cfb902adf647a4b6f2e6", "type": "function", "function": {"name": "get_current_weather", "arguments": "{\"location\": \"广州\"}"}}}
>-[TOOL_RESP]>> 广州今天下大雨。 
雨幕洒羊城，水珠舞街巷，
伞开花如海，滋润春色长。



[{'role': 'system',
  'content': '你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n**你的任务是:**\n根据广州今天的天气写一首4句歌词\n'},
 {'role': 'user', 'content': '请开始'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'index': 0,
    'id': 'call_b9cfb902adf647a4b6f2e6',
    'type': 'function',
    'function': {'name': 'get_current_weather',
     'arguments': '{"location": "广州"}'}}]},
 {'role': 'tool', 'name': 'get_current_weather', 'content': '广州今天下大雨。 '},
 {'role': 'assistant', 'content': '雨幕洒羊城，水珠舞街巷，\n伞开花如海，滋润春色长。'}]

In [12]:
d.output

'雨幕洒羊城，水珠舞街巷，\n伞开花如海，滋润春色长。\n\n'

### 单轮写作 + 多轮对话

In [13]:
d.chat("改为8句吧")

雨帘密织羊城天，湿了红尘润了田，
珠江水面泛轻烟，朦胧楼影入画卷。
行人匆匆伞下聚，花朵含珠更娇艳，
春雨绵绵情意浓，洗尽铅华现真颜。
街头小巷故事多，雨声伴奏唱生活，
望穿雨幕待晴空，广州雨中别样红。



[{'role': 'system',
  'content': '你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n**你的任务是:**\n根据广州今天的天气写一首4句歌词\n'},
 {'role': 'user', 'content': '请开始'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'index': 0,
    'id': 'call_b9cfb902adf647a4b6f2e6',
    'type': 'function',
    'function': {'name': 'get_current_weather',
     'arguments': '{"location": "广州"}'}}]},
 {'role': 'tool', 'name': 'get_current_weather', 'content': '广州今天下大雨。 '},
 {'role': 'assistant', 'content': '雨幕洒羊城，水珠舞街巷，\n伞开花如海，滋润春色长。'},
 {'role': 'user', 'content': '改为8句吧'},
 {'role': 'assistant',
  'content': '雨帘密织羊城天，湿了红尘润了田，\n珠江水面泛轻烟，朦胧楼影入画卷。\n行人匆匆伞下聚，花朵含珠更娇艳，\n春雨绵绵情意浓，洗尽铅华现真颜。\n街头小巷故事多，雨声伴奏唱生活，\n望穿雨幕待晴空，广州雨中别样红。'}]

In [14]:
d.state.messages

[{'role': 'system',
  'content': '你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n**你的任务是:**\n根据广州今天的天气写一首4句歌词\n'},
 {'role': 'user', 'content': '请开始'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'index': 0,
    'id': 'call_b9cfb902adf647a4b6f2e6',
    'type': 'function',
    'function': {'name': 'get_current_weather',
     'arguments': '{"location": "广州"}'}}]},
 {'role': 'tool', 'name': 'get_current_weather', 'content': '广州今天下大雨。 '},
 {'role': 'assistant', 'content': '雨幕洒羊城，水珠舞街巷，\n伞开花如海，滋润春色长。'},
 {'role': 'user', 'content': '改为8句吧'},
 {'role': 'assistant',
  'content': '雨帘密织羊城天，湿了红尘润了田，\n珠江水面泛轻烟，朦胧楼影入画卷。\n行人匆匆伞下聚，花朵含珠更娇艳，\n春雨绵绵情意浓，洗尽铅华现真颜。\n街头小巷故事多，雨声伴奏唱生活，\n望穿雨幕待晴空，广州雨中别样红。'}]

In [23]:
from textlong.llm import fake_llm
d.chat("这是古典诗歌吗？", llm=qwen, k=1)

不是的，任务要求根据广州今天的天气写一首四句歌词，这更接近于现代歌词创作而非古典诗歌。古典诗歌通常有特定的格律、押韵规则及意象运用，而根据天气创作歌词则更为自由灵活，偏向于现代歌曲的表达方式。



[{'role': 'system',
  'content': '你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n**你的任务是:**\n根据广州今天的天气写一首4句歌词\n'},
 {'role': 'user', 'content': '请开始'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'index': 0,
    'id': 'call_b9cfb902adf647a4b6f2e6',
    'type': 'function',
    'function': {'name': 'get_current_weather',
     'arguments': '{"location": "广州"}'}}]},
 {'role': 'tool', 'name': 'get_current_weather', 'content': '广州今天下大雨。 '},
 {'role': 'assistant', 'content': '雨幕洒羊城，水珠舞街巷，\n伞开花如海，滋润春色长。'},
 {'role': 'user', 'content': '这是古典诗歌吗？'},
 {'role': 'assistant', 'content': '这是一个模拟调用!'},
 {'role': 'user', 'content': '这是古典诗歌吗？'},
 {'role': 'assistant',
  'content': '不是的，任务要求根据广州今天的天气写一首四句歌词，这更接近于现代歌词创作而非古典诗歌。古典诗歌通常有特定的格律、押韵规则及意象运用，而根据天气创作歌词则更为自由灵活，偏向于现代歌曲的表达方式。'}]

In [22]:
d.state.messages

[{'role': 'system',
  'content': '你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n**你的任务是:**\n根据广州今天的天气写一首4句歌词\n'},
 {'role': 'user', 'content': '请开始'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'index': 0,
    'id': 'call_b9cfb902adf647a4b6f2e6',
    'type': 'function',
    'function': {'name': 'get_current_weather',
     'arguments': '{"location": "广州"}'}}]},
 {'role': 'tool', 'name': 'get_current_weather', 'content': '广州今天下大雨。 '},
 {'role': 'assistant', 'content': '雨幕洒羊城，水珠舞街巷，\n伞开花如海，滋润春色长。'},
 {'role': 'user', 'content': '改为8句吧'},
 {'role': 'assistant',
  'content': '雨帘密织羊城天，湿了红尘润了田，\n珠江水面泛轻烟，朦胧楼影入画卷。\n行人匆匆伞下聚，花朵含珠更娇艳，\n春雨绵绵情意浓，洗尽铅华现真颜。\n街头小巷故事多，雨声伴奏唱生活，\n望穿雨幕待晴空，广州雨中别样红。'},
 {'role': 'user', 'content': '类似西方的诗歌那样风格'},
 {'role': 'assistant', 'content': '这是一个模拟调用!'},
 {'role': 'user', 'content': '类似西方的诗歌那样风格'},
 {'role': 'assistant', 'content': '这是一个模拟调用!'},
 {'role': 'user', 'content': '类似西方的诗歌那样风格'},
 {'role': 'assistant', 'content': '这是一个

### 提纲 + 扩写

In [1]:
from textlong.llm import qwen, fake_llm
from textlong.desk import Desk

d = Desk(qwen)

In [2]:
d.write({"task": "帮我写一首关于兔子的儿歌，两段即可，第一段些兔子有哪些朋友，第二段些和朋友们一起玩（要求提到具体的朋友）"})
d.output

# 兔子的欢乐儿歌

## 第一段：兔子的朋友圈
<OUTLINE>
扩写摘要: 描述森林中兔子的朋友们，强调友谊的多样性和温馨。

扩写要求：
- 预估字数：30字
- 创意要点：列举三到四种不同动物作为兔子的朋友，每种动物简要特征描述。
- 创作思路：通过生动活泼的语言，展现一个和谐共处的动物小世界。
- 实体名称：兔子、松鼠、小鸟、小鹿
</OUTLINE>

## 第二段：快乐的玩耍时光
<OUTLINE>
扩写摘要: 讲述兔子与它的朋友们在森林中的趣味游戏，突出团队合作与快乐分享。

扩写要求：
- 预估字数：30字
- 创意要点：描述一个具体的游戏场景，比如躲猫猫或赛跑，体现每个朋友的特长。
- 创作思路：利用节奏明快的句子，传达出游戏的紧张与欢笑，以及友情的加深。
- 实体名称：兔子（快速跳跃）、松鼠（灵巧爬树）、小鸟（高空飞翔）、小鹿（敏捷奔跑）
</OUTLINE>



'# 兔子的欢乐儿歌\n\n\n## 第一段：兔子的朋友圈\n\n<OUTLINE>\n扩写摘要: 描述森林中兔子的朋友们，强调友谊的多样性和温馨。\n\n扩写要求：\n- 预估字数：30字\n- 创意要点：列举三到四种不同动物作为兔子的朋友，每种动物简要特征描述。\n- 创作思路：通过生动活泼的语言，展现一个和谐共处的动物小世界。\n- 实体名称：兔子、松鼠、小鸟、小鹿\n</OUTLINE>\n\n\n## 第二段：快乐的玩耍时光\n\n<OUTLINE>\n扩写摘要: 讲述兔子与它的朋友们在森林中的趣味游戏，突出团队合作与快乐分享。\n\n扩写要求：\n- 预估字数：30字\n- 创意要点：描述一个具体的游戏场景，比如躲猫猫或赛跑，体现每个朋友的特长。\n- 创作思路：利用节奏明快的句子，传达出游戏的紧张与欢笑，以及友情的加深。\n- 实体名称：兔子（快速跳跃）、松鼠（灵巧爬树）、小鸟（高空飞翔）、小鹿（敏捷奔跑）\n</OUTLINE>\n\n'

In [3]:
for doc in d.state.markdown.documents:
    print(doc.metadata['id'])

1779-646-000
1779-007-001
1779-437-002
1779-220-003
1779-908-004
1779-581-005
1779-656-006


In [4]:
d.from_outline(llm=qwen)
d.output

>-[INFO]>> 执行扩写任务：
扩写摘要: 描述森林中兔子的朋友们，强调友谊的多样性和温馨。

扩写要求：
- 预估字数：30字
- 创意要点：列举三到四种不同动物作为兔子的朋友，每种动物简要特征描述。
- 创作思路：通过生动活泼的语言，展现一个和谐共处的动物小世界。
- 实体名称：兔子、松鼠、小鸟、小鹿
## 第一段：兔子的朋友圈

在翠绿的森林深处，兔子拥有一群快乐的朋友：机敏的松鼠，总能找到最香的坚果；歌唱家小鸟，每天清晨以旋律唤醒太阳；还有温柔的小鹿，它们一起在林间轻盈跳跃。这个多彩的圈子，编织着友情的温暖篇章。

>-[INFO]>> 执行扩写任务：
扩写摘要: 讲述兔子与它的朋友们在森林中的趣味游戏，突出团队合作与快乐分享。

扩写要求：
- 预估字数：30字
- 创意要点：描述一个具体的游戏场景，比如躲猫猫或赛跑，体现每个朋友的特长。
- 创作思路：利用节奏明快的句子，传达出游戏的紧张与欢笑，以及友情的加深。
- 实体名称：兔子（快速跳跃）、松鼠（灵巧爬树）、小鸟（高空飞翔）、小鹿（敏捷奔跑）
## 第二段：快乐的玩耍时光

在葱郁森林深处，兔子与好友共舞乐章，
躲猫猫游戏正酣，欢声笑语溢满林间走廊。
兔子快速跳跃，灵活躲藏于繁花之后，
松鼠灵巧爬树，枝头变戏法，树叶沙沙响不休。
小鸟高空飞翔，扮演天空的眼睛，俯瞰着追逐的踪迹，
小鹿敏捷奔跑，在阳光下划出一道道光影交错的奇迹。
团队协作显神威，友谊在嬉戏中更加坚不可摧，
这一天，森林回荡着它们快乐的歌，纯粹又明媚。



'# 兔子的欢乐儿歌\n\n\n## 第一段：兔子的朋友圈\n\n\n在翠绿的森林深处，兔子拥有一群快乐的朋友：机敏的松鼠，总能找到最香的坚果；歌唱家小鸟，每天清晨以旋律唤醒太阳；还有温柔的小鹿，它们一起在林间轻盈跳跃。这个多彩的圈子，编织着友情的温暖篇章。\n\n\n## 第二段：快乐的玩耍时光\n\n\n在葱郁森林深处，兔子与好友共舞乐章，\n躲猫猫游戏正酣，欢声笑语溢满林间走廊。\n兔子快速跳跃，灵活躲藏于繁花之后，\n松鼠灵巧爬树，枝头变戏法，树叶沙沙响不休。\n小鸟高空飞翔，扮演天空的眼睛，俯瞰着追逐的踪迹，\n小鹿敏捷奔跑，在阳光下划出一道道光影交错的奇迹。\n团队协作显神威，友谊在嬉戏中更加坚不可摧，\n这一天，森林回荡着它们快乐的歌，纯粹又明媚。\n\n'

In [5]:
d.state.from_outline

{'1779-220-003': [{'role': 'system',
   'content': '你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n3. 你必须根据已有提纲扩写，不要修改提纲中对扩写的要求和限定，不要额外发挥\n**已有文字草稿如下:**\n```markdown\n# 兔子的欢乐儿歌\n\n\n## 第一段：兔子的朋友圈\n\n&lt;&lt;&lt;YOUR_TEXT&gt;&gt;&gt;\n\n\n## 第二段：快乐的玩耍时光\n\n...\n\n```\n\n**请你按照如下扩写任务要求生成一段文字，使其适合替换上面文字草稿中`<<<YOUR_TEXT>>>`所在位置:**\n```markdown\n扩写摘要: 描述森林中兔子的朋友们，强调友谊的多样性和温馨。\n\n扩写要求：\n- 预估字数：30字\n- 创意要点：列举三到四种不同动物作为兔子的朋友，每种动物简要特征描述。\n- 创作思路：通过生动活泼的语言，展现一个和谐共处的动物小世界。\n- 实体名称：兔子、松鼠、小鸟、小鹿\n```\n'},
  {'role': 'user', 'content': '请开始'},
  {'role': 'assistant',
   'content': '## 第一段：兔子的朋友圈\n\n在翠绿的森林深处，兔子拥有一群快乐的朋友：机敏的松鼠，总能找到最香的坚果；歌唱家小鸟，每天清晨以旋律唤醒太阳；还有温柔的小鹿，它们一起在林间轻盈跳跃。这个多彩的圈子，编织着友情的温暖篇章。'}],
 '1779-656-006': [{'role': 'system',
   'content': '你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n3. 你必须根据已有提纲扩写，不要修改提纲中对扩写的要求和限定，不要额外发挥\n**已有文字草稿如下:**\n```markdown\n# 兔子的欢乐儿歌\n\n\n## 第一段：兔子的朋友圈\n\n...\n\n## 第二段：快乐的玩耍时光\n\n&lt;&lt;&lt;YOUR_TEXT&gt;&gt;&gt;\

In [12]:
print(d.state.markdown.text)

# 兔子的欢乐儿歌


## 第一段：兔子的朋友圈

<OUTLINE>
扩写摘要: 描述森林中兔子的朋友们，强调友谊的多样性和温馨。

扩写要求：
- 预估字数：30字
- 创意要点：列举三到四种不同动物作为兔子的朋友，每种动物简要特征描述。
- 创作思路：通过生动活泼的语言，展现一个和谐共处的动物小世界。
- 实体名称：兔子、松鼠、小鸟、小鹿
</OUTLINE>


## 第二段：快乐的玩耍时光

<OUTLINE>
扩写摘要: 讲述兔子与它的朋友们在森林中的趣味游戏，突出团队合作与快乐分享。

扩写要求：
- 预估字数：30字
- 创意要点：描述一个具体的游戏场景，比如躲猫猫或赛跑，体现每个朋友的特长。
- 创作思路：利用节奏明快的句子，传达出游戏的紧张与欢笑，以及友情的加深。
- 实体名称：兔子（快速跳跃）、松鼠（灵巧爬树）、小鸟（高空飞翔）、小鹿（敏捷奔跑）
</OUTLINE>


